In [1]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from glob import glob
import time
import copy

from tqdm import tqdm_notebook as tq
from mobilenetv3_pytorch import mobilenetv3

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.quantization import convert
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms, utils

import os

%matplotlib inline

In [2]:
# !pip3 install --user facenet-pytorch

In [3]:
from facenet_pytorch import MTCNN

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [5]:
root = 'MeGlass_120x120'
data_dir = 'MeGlass_faces'

batch_size = 128
epochs = 8
workers = 8

glasses_images_paths = glob(f"{root}/glasses/*.jpg")
no_glasses_images_paths = glob(f"{root}/no_glasses/*.jpg")

In [6]:
%%time
mtcnn = MTCNN(128, device=device)

for p in tq(glasses_images_paths):
    image_id = p.split('/')[-1]
    image = cv2.imread(p)
    mtcnn(image, save_path=f"{data_dir}/glasses/{image_id}")

for p in tq(no_glasses_images_paths):
    image_id = p.split('/')[-1]
    image = cv2.imread(p)
    mtcnn(image, save_path=f"{data_dir}/no_glasses/{image_id}")

del mtcnn


CPU times: user 32min 26s, sys: 17.3 s, total: 32min 43s
Wall time: 8min 59s


In [7]:
tr = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]),
    ])
data = datasets.ImageFolder(data_dir, transform=tr)

In [8]:
img_inds = np.arange(len(data))
np.random.shuffle(img_inds)

train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    data,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    data,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

dataloaders = {
    'train' : train_loader,
    'val' : val_loader
}

dataset_sizes = {
    'train' : len(train_inds),
    'val' : len(val_inds)
}

class_names = data.classes

In [9]:
net = mobilenetv3.mobilenetv3_small()

mod = nn.Sequential(
    net.features[:10],
    net.conv,
    net.avgpool,
    nn.Flatten(),
    nn.Linear(in_features=576, out_features=2)
)

# mod.load_state_dict(torch.load('best.pth'))
mod.to(device)

optimizer = optim.SGD(mod.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
loss_fn = nn.CrossEntropyLoss()

In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f"Training completed in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    
    print(f"Best val Acc: {best_acc:4f}")

    model.load_state_dict(best_model_wts)
    return model

In [11]:
best_model = train_model(mod, loss_fn, optimizer, scheduler, num_epochs=10)

Epoch 0/9
----------
train Loss: 0.1722 Acc: 0.9320
val Loss: 0.0341 Acc: 0.9902

Epoch 1/9
----------
train Loss: 0.0217 Acc: 0.9945
val Loss: 0.0179 Acc: 0.9946

Epoch 2/9
----------
train Loss: 0.0151 Acc: 0.9958
val Loss: 0.0129 Acc: 0.9960

Epoch 3/9
----------
train Loss: 0.0098 Acc: 0.9976
val Loss: 0.0115 Acc: 0.9960

Epoch 4/9
----------
train Loss: 0.0084 Acc: 0.9976
val Loss: 0.0110 Acc: 0.9968

Epoch 5/9
----------
train Loss: 0.0069 Acc: 0.9983
val Loss: 0.0126 Acc: 0.9964

Epoch 6/9
----------
train Loss: 0.0053 Acc: 0.9986
val Loss: 0.0114 Acc: 0.9969

Epoch 7/9
----------
train Loss: 0.0039 Acc: 0.9988
val Loss: 0.0108 Acc: 0.9971

Epoch 8/9
----------
train Loss: 0.0027 Acc: 0.9994
val Loss: 0.0103 Acc: 0.9970

Epoch 9/9
----------
train Loss: 0.0025 Acc: 0.9995
val Loss: 0.0096 Acc: 0.9971

Training completed in 2m 2s
Best val Acc: 0.997127


In [12]:
best_model.cpu()

torch.save(best_model.state_dict(), 'best.pth')

In [13]:
!du -sh best.pth

2,0M	best.pth
